In [1]:
# ----- Check if GPU is connected ----- # 
gpu_info = !nvidia-smi -L
gpu_info = "\n".join(gpu_info)
if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU")
else:
    print(gpu_info)


GPU 0: Tesla T4 (UUID: GPU-5364bc84-a97f-3b67-bc60-bd038ed703d9)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **DialogSumm**

In [3]:
!git clone https://github.com/PranjaliJain/dialogsum_trained.git

Cloning into 'dialogsum_trained'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 173 (delta 86), reused 112 (delta 38), pack-reused 9
Receiving objects: 100% (173/173), 8.88 MiB | 29.15 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [4]:
%cd /content/dialogsum_trained
# !git stash
#!git pull 
#!git status

/content/dialogsum_trained


In [5]:
!pip install datasets
!pip install transformers
!pip install rouge_score
!pip install --user -U nltk

     |████████████████████████████████| 342 kB 9.4 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.0 MB/s 
     |████████████████████████████████| 136 kB 74.1 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
     |████████████████████████████████| 127 kB 76.6 MB/s 
     |████████████████████████████████| 271 kB 72.4 MB/s 
     |████████████████████████████████| 144 kB 72.6 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 8.2 MB/s 


In [6]:
import json
from datasets import load_metric,Dataset,DatasetDict,load_dataset
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import AutoTokenizer
# import os
from torch import nn 
import torch

import nltk
nltk.download('punkt')

import sys
from google.colab import drive
import pandas as pd
import numpy as np
import huggingface_hub
import matplotlib.pyplot as plt
import gc


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# ----- Check if GPU is connected ----- # 
gpu_info = !nvidia-smi -L
gpu_info = "\n".join(gpu_info)
if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU")
else:
    print(gpu_info)


GPU 0: Tesla T4 (UUID: GPU-5364bc84-a97f-3b67-bc60-bd038ed703d9)


In [8]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
model_checkpoint = "facebook/bart-large"
metric = load_metric("rouge")

TEST_SUMMARY_ID = 1


def transform_single_dialogsumm_file(file):
    data = open(file,"r").readlines()
    result = {"fname":[],"summary":[],"dialogue":[]}
    for i in data:
        d = json.loads(i)
        for j in d.keys():
            if j in result.keys():
                result[j].append(d[j])
    return Dataset.from_dict(result)

def transform_test_file(file):
    data = open(file,"r").readlines()
    result = {"fname":[],"summary%d"%TEST_SUMMARY_ID:[],"dialogue":[]}
    for i in data:
        d = json.loads(i)
        for j in d.keys():
            if j in result.keys():
                result[j].append(d[j])
    
    result["summary"] = result["summary%d"%TEST_SUMMARY_ID]
    return Dataset.from_dict(result)

def transform_dialogsumm_to_huggingface_dataset(train,validation,test):
    train = transform_single_dialogsumm_file(train)
    validation = transform_single_dialogsumm_file(validation)
    test = transform_test_file(test)
    return DatasetDict({"train":train,"validation":validation,"test":test})


def freeze_params(model: nn.Module):
    """Set requires_grad=False for each of model.parameters()"""
    for par in model.parameters():
        par.requires_grad = False

def freeze_embeds(model):
    """Freeze token embeddings and positional embeddings for BART and PEGASUS, just token embeddings for t5."""
    model_type = model.config.model_type
    if model_type == "t5":
        freeze_params(model.shared)
        for d in [model.encoder, model.decoder]:
            freeze_params(d.embed_tokens)
    else:
        freeze_params(model.model.shared)
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [10]:
raw_datasets = transform_dialogsumm_to_huggingface_dataset("/content/dialogsum_trained/DialogSum_Data/dialogsum.train.jsonl","/content/dialogsum_trained/DialogSum_Data/dialogsum.dev.jsonl","/content/dialogsum_trained/DialogSum_Data/dialogsum.test.jsonl")

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'


model = BartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

freeze_embeds(model)

max_input_length = 256
max_target_length = 128

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
## use batch_size = 1 to get time around 8 hrs with GPU
## use batch_size = 16 to get time around 4 hrs with GPU - but it goes out of memory
batch_size = 4
args = Seq2SeqTrainingArguments(
    "BART-LARGE-DIALOGSUM",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch",
    # metric_for_best_model="eval_rouge1",
    # greater_is_better=True,
    # seed=42,
    # generation_max_length=max_target_length,
    push_to_hub=False,
    #output_dir = '/content/drive/MyDrive/project/results/dialogsumm-bart'
    #logging_dir = '/content/drive/MyDrive/project/results/dialogsumm-bart'
)


In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, fname, dialogue. If summary, fname, dialogue are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12460
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 9345


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.227800,1.132623,41.544000,21.047700,36.502200,38.438400,19.938000
2,1.015900,1.107988,43.113500,22.944400,38.501400,40.111800,19.876000
3,0.887500,1.096485,42.263900,21.763400,37.583600,39.225900,19.908000


Saving model checkpoint to BART-LARGE-DIALOGSUM/checkpoint-500
Configuration saved in BART-LARGE-DIALOGSUM/checkpoint-500/config.json
Model weights saved in BART-LARGE-DIALOGSUM/checkpoint-500/pytorch_model.bin
tokenizer config file saved in BART-LARGE-DIALOGSUM/checkpoint-500/tokenizer_config.json
Special tokens file saved in BART-LARGE-DIALOGSUM/checkpoint-500/special_tokens_map.json
Saving model checkpoint to BART-LARGE-DIALOGSUM/checkpoint-1000
Configuration saved in BART-LARGE-DIALOGSUM/checkpoint-1000/config.json
Model weights saved in BART-LARGE-DIALOGSUM/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in BART-LARGE-DIALOGSUM/checkpoint-1000/tokenizer_config.json
Special tokens file saved in BART-LARGE-DIALOGSUM/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to BART-LARGE-DIALOGSUM/checkpoint-1500
Configuration saved in BART-LARGE-DIALOGSUM/checkpoint-1500/config.json
Model weights saved in BART-LARGE-DIALOGSUM/checkpoint-1500/pytorch_model.bin
tok

TrainOutput(global_step=9345, training_loss=1.0795598094893242, metrics={'train_runtime': 3744.2886, 'train_samples_per_second': 9.983, 'train_steps_per_second': 2.496, 'total_flos': 1.9283523178266624e+16, 'train_loss': 1.0795598094893242, 'epoch': 3.0})

In [13]:

out = trainer.predict(tokenized_datasets["test"],num_beams=5)
predictions, labels ,metric= out
print(metric)

The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, summary1, fname, dialogue. If summary, summary1, fname, dialogue are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 500
  Batch size = 4


{'test_loss': 1.3360599279403687, 'test_rouge1': 41.5981, 'test_rouge2': 18.7302, 'test_rougeL': 35.8956, 'test_rougeLsum': 37.8259, 'test_gen_len': 19.866, 'test_runtime': 73.2878, 'test_samples_per_second': 6.822, 'test_steps_per_second': 1.706}


In [14]:
decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after e ach sentence
decoded_preds = [" ".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
decoded_labels = [" ".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

In [15]:
# output summaries on test set
with open("test_output.txt","w") as f: 
    for i in decoded_preds:
        print(i)
        f.write(i.replace("\n","")+"\n")

#Person1# asks Ms. Dawson to take a dictation for him
#Person2# got stuck in traffic again. #Person1# suggests #Person
#Person1# tells Kate Masha and Hero are getting divorced. Kate is surprised
#Person1# wishes Brian a happy birthday and dances with him at the party
#Person1# and #Person2# are in the Olympic stadium. #Person
#Person1# tells #Person2# #Person3# is quitting
#Person2# feels itchy and #Person1#Person3#
#Person2# checks out and finds the bill has been added to someone else's
#Person1# asks Steven for help because his wife is going to divorce him.
#Person2# thinks of Abraham Lincoln as a man or woman of sound character.
#Person1# tells #Person2# the north of China are experiencing severe sand
Francis gets a remote car as a birthday gift from #Person2#. Francis
Tony tells Steven he got caught cheating and feels ashamed. Steven advises him to study hard
#Person1# is in a hurry to catch the nine-thirty train.
#Person1# asks #Person2# for advice on adjusting life. #Person
#Pers

In [23]:
model.save_pretrained('./saved_model/')
tokenizer.save_pretrained('./saved_model/')

Configuration saved in ./saved_model/config.json
Model weights saved in ./saved_model/pytorch_model.bin
tokenizer config file saved in ./saved_model/tokenizer_config.json
Special tokens file saved in ./saved_model/special_tokens_map.json


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/merges.txt',
 './saved_model/added_tokens.json')

# **TweetSumm**

In [27]:
%cd /content/TweetSumm_trained


/content/TweetSumm_trained


In [ ]:
!git clone https://github.com/PranjaliJain/TweetSumm_trained.git

In [28]:
from tweet_sum_processor import TweetSumProcessor

In [30]:
model_name = './../dialogsum_trained/saved_model/'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

Didn't find file ./../dialogsum_trained/saved_model/added_tokens.json. We won't load it.
loading file ./../dialogsum_trained/saved_model/vocab.json
loading file ./../dialogsum_trained/saved_model/merges.txt
loading file None
loading file ./../dialogsum_trained/saved_model/special_tokens_map.json
loading file ./../dialogsum_trained/saved_model/tokenizer_config.json
loading configuration file ./../dialogsum_trained/saved_model/config.json
Model config BartConfig {
  "_name_or_path": "./saved_model/",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  

In [31]:
# ----- Metric
metric = load_metric("rouge")

# ---- Freeze parameters

def freeze_params(model: nn.Module):
    """Set requires_grad=False for each of model.parameters()"""
    for par in model.parameters():
        par.requires_grad = False

def freeze_embeds(model):
    """Freeze token embeddings and positional embeddings for BART and PEGASUS, just token embeddings for t5."""
    model_type = model.config.model_type
    if model_type == "t5":
        freeze_params(model.shared)
        for d in [model.encoder, model.decoder]:
            freeze_params(d.embed_tokens)
    else:
        freeze_params(model.model.shared)
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)

freeze_embeds(model)

# ----- Reading in the Dataset
raw_datasets = load_dataset('csv', data_files={'train': '/content/TweetSumm_trained/data/tweetsum_train.csv',
                                          'valid': '/content/TweetSumm_trained/data/tweetsum_valid.csv',
                                          'test': '/content/TweetSumm_trained/data/tweetsum_test.csv'})

Using custom data configuration default-3760b9b9fac89e08


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3760b9b9fac89e08/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
max_input_length = 256
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(examples["inputs"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summaries"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
batch_size = 1
args = Seq2SeqTrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    output_dir = '/content/drive/MyDrive/CIS6930_final/results/bart', 
    logging_dir = '/content/drive/MyDrive/CIS6930_final/logs/bart'
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# **NO fine-tuning on TweetSumm**

In [35]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# trainer.train()

Using amp half precision backend


In [36]:
# --------------------- # 
#    TEST EVALUATION    #
# --------------------- #

out = trainer.predict(tokenized_datasets["test"])
generated_summaries = []
for i in range(0, 110): 
  generated_summaries.append(tokenizer.decode(out[0][i], skip_special_tokens =  True))
ground_truth = tokenized_datasets["test"]["summaries"]
conversation = tokenized_datasets["test"]["inputs"]

The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: inputs, summaries. If inputs, summaries are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 110
  Batch size = 1


In [37]:
out.metrics

{'test_gen_len': 20.0,
 'test_loss': 3.859863519668579,
 'test_rouge1': 20.3368,
 'test_rouge2': 6.4265,
 'test_rougeL': 16.5027,
 'test_rougeLsum': 18.4148,
 'test_runtime': 40.5731,
 'test_samples_per_second': 2.711,
 'test_steps_per_second': 2.711}

In [ ]:
P, R, F1 = score(generated_summaries, ground_truth, lang="en", verbose=True)

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level precision score: {P.mean():.3f}")
print(f"System level recall score: {R.mean():.3f}")

# **Fine-tuning on TweetSumm**

In [38]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: inputs, summaries. If inputs, summaries are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 869
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 2607


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.379300,2.108410,32.422400,14.249700,27.878600,30.325700,20.000000
2,1.732400,2.064653,34.135200,15.376200,29.347500,32.141700,20.000000
3,1.416900,2.076144,36.076100,16.923800,31.533700,34.217600,20.000000


Saving model checkpoint to /content/drive/MyDrive/CIS6930_final/results/bart/checkpoint-500
Configuration saved in /content/drive/MyDrive/CIS6930_final/results/bart/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/CIS6930_final/results/bart/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/CIS6930_final/results/bart/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/CIS6930_final/results/bart/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: inputs, summaries. If inputs, summaries are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 108
  Batch size = 1
Saving model checkpoint to /content/drive/MyDrive/CIS6930_final/results/bart/checkpoint-1000
Configuration saved i

TrainOutput(global_step=2607, training_loss=1.798732568808886, metrics={'train_runtime': 1026.9638, 'train_samples_per_second': 2.539, 'train_steps_per_second': 2.539, 'total_flos': 1277508162945024.0, 'train_loss': 1.798732568808886, 'epoch': 3.0})

In [39]:
# --------------------- # 
#    TEST EVALUATION    #
# --------------------- #

out = trainer.predict(tokenized_datasets["test"])
generated_summaries = []
for i in range(0, 110): 
  generated_summaries.append(tokenizer.decode(out[0][i], skip_special_tokens =  True))
ground_truth = tokenized_datasets["test"]["summaries"]
conversation = tokenized_datasets["test"]["inputs"]

The following columns in the test set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: inputs, summaries. If inputs, summaries are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 110
  Batch size = 1


In [40]:
out.metrics

{'test_gen_len': 20.0,
 'test_loss': 2.0977628231048584,
 'test_rouge1': 34.1796,
 'test_rouge2': 14.9658,
 'test_rougeL': 30.0137,
 'test_rougeLsum': 32.0041,
 'test_runtime': 40.3824,
 'test_samples_per_second': 2.724,
 'test_steps_per_second': 2.724}

In [ ]:
P, R, F1 = score(generated_summaries, ground_truth, lang="en", verbose=True)

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level precision score: {P.mean():.3f}")
print(f"System level recall score: {R.mean():.3f}")